In [1]:
import atomium
import math
import mdtraj as md
import nglview
import pybel as bel
import os
from tqdm import tqdm
from rmsd import *
import numpy as np

_ColormakerRegistry()

In [3]:
def trans_rot(P,Q):
    
    p = P.mean(axis = 0)
    q = Q.mean(axis = 0)
    T = q-p
    P += T
    
    R = kabsch(P,Q)
    return np.concatenate(T, np.ndarray.flatten(R))
    